In [111]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G52   forever...
1     Yeah   G15   it was such a change! I  G40   s...
2       G13  And I was so glad that all of the meas...
3     Ben and I had been saying for the past two we...
4     Of course   G20  that is what happened for th...
Name: sentence, dtype: object

In [47]:

class data():
    def __init__(self):
        self.data = []
        
class mini_data():
    def __init__(self):
        self.mini_data = []
        
d = data()

'''separate words in sentences and store them in lists'''
def read_data(sentence):
    md = mini_data()
    sentence = sentence.lower()
    #content = sentence.split()
    content = re.findall(r"[\w']+|[.,!?;]", sentence)
    new = md.mini_data + content
    d.data.append(new)

df_data['sentence'].apply(read_data)
training_data = np.array(d.data)
print(training_data[0])

print('training_data length: ' + str(len(training_data)))

avrgs = [len(x) for x in training_data]
print('longest sentence: ' + str(np.max(avrgs)))
print('shortest sentence: ' + str(np.min(avrgs)))
print('avrg sentence length: ' + str(np.mean(avrgs)))
print('standard deviation of sentence length: ' + str(np.std(avrgs)))

'''sentence length will be the average length plus the standard deviation'''
sentence_length = round(np.mean(avrgs) + np.std(avrgs))
print('sentence length: ' + str(sentence_length))


['after', 'waiting', 'what', 'seemed', 'like', 'g52', 'forever', 'to', 'get', 'into', 'the', 'room', '.', 'we', 'g28', 'finally', 'got', 'to', 'see', 'our', 'little', 'one', 'for', 'the', 'first', 'time', 'in', '7', 'weeks', 'g15', 'g44', 'and', 'wow', 'what', 'a', 'change', 'it', 'was', '!']
training_data length: 874
longest sentence: 160
shortest sentence: 1
avrg sentence length: 29.1830663616
standard deviation of sentence length: 17.4226866681
sentence length: 47.0


In [139]:
import collections

def filter_gestures(word):
    ges = re.search(r'g\d{1,2}',word)
    if ges is None:
        return True
    else:
        return False

def build_dictionary(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)+1
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

'''make the list of list of words into list of words'''
words = [item for sublist in training_data for item in sublist]

'''filter the gesture annotations'''
words = [x for x in words if filter_gestures(x)]

'''count the unique appearance of words and assign them a number'''
dictionary, reverse_dictionary = build_dictionary(words)

#print(dictionary)
vocab_size = len(dictionary)
print(vocab_size)

3214


In [134]:
import tensorflow as tf
from tensorflow.contrib import rnn

#sequence length
time_steps=4

#could be any number
num_units=20

#self explanatory
input_size=1

#we need to learn about this
learning_rate=0.001

#57 getsure types + no gesture
n_classes=58
no_gesture_index = 57

#tbd
batch_size=100

In [110]:

train_input = []
train_output = []

def create_input_output(flat_data):
    
    prev_gesture = False
    
    single_input = []
    single_output = [0]*n_classes    
    
    for i in range(len(flat_data)):
    
        count = 0
        extra_index = 0
        words = flat_data[i+extra_index:i+time_steps+extra_index]
        
        if len(words) != time_steps:
            break
        
        while count < time_steps + extra_index:
            
            word = words[count]   
            gesture = re.search(r'g\d{1,2}', word)

            if gesture is None:

                num = dictionary[word]
                single_input.append(num)
                
                if count+1 == len(words) and sum(single_output) == 0:
                    single_output[no_gesture_index] = 1

            elif gesture is not None:

                if count+1 == len(words):
                    index_gesture = int(word[1:])
                    single_output[index_gesture] = 1
                
                new_word = flat_data[i+time_steps+extra_index]
                words.append(new_word)
                extra_index = extra_index + 1
                
            count = count+1
                
            
        train_input.append(single_input)
        train_output.append(single_output)  

        single_input = []
        single_output = [0]*n_classes  
    
    return 'Done!'
            
flat_training_data = [item for sublist in training_data for item in sublist]
create_input_output(flat_training_data)
                       
print(len(flat_training_data))                 
print(len(train_input))
print(len(train_output))

25506
25503
25503
[259, 1, 1, 1]


In [128]:
#Random sampling training and test data

_TRAINING_INS_COUNT = 20500
train_indexes = random.sample(range(0, len(train_input)), _TRAINING_INS_COUNT)
test_indexes = [i for i in range(0, len(train_input)) if not i in train_indexes]


#creating training data
_TRAINING_INPUT = np.array(train_input)
_TRAINING_INPUT = _TRAINING_INPUT[train_indexes]
_TRAINING_OUTPUT = np.array(train_output)
_TRAINING_OUTPUT = _TRAINING_OUTPUT[train_indexes]


#creating testing data
_TESTING_INPUT = np.array(train_input)
_TESTING_INPUT = _TESTING_INPUT[test_indexes]
_TESTING_OUTPUT = np.array(train_output)
_TESTING_OUTPUT = _TESTING_OUTPUT[test_indexes]


20500
5003


In [130]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,input_size])
print(x)
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,input_size] to "time_steps" number of [batch_size,input_size] tensors
input=tf.unstack(x ,time_steps,1)
print(input)

Tensor("Placeholder_2:0", shape=(?, 4, 1), dtype=float32)
[<tf.Tensor 'unstack_1:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_1:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_1:2' shape=(?, 1) dtype=float32>, <tf.Tensor 'unstack_1:3' shape=(?, 1) dtype=float32>]


In [131]:
# defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)

with tf.variable_scope("rnn", reuse=None):
    outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")
    print(outputs[-1])
    
    #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
    prediction=tf.matmul(outputs[-1],out_weights)+out_bias
    print(prediction)

    #loss_function
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    #optimization
    opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    #model evaluationmatmul
    correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
Tensor("rnn/add:0", shape=(?, 58), dtype=float32)


In [137]:
#initialize variables
batch_init = 0
limit = batch_init + batch_size
init=tf.global_variables_initializer()

with tf.Session() as sess:
    
    sess.run(init)
    iter=1
    while iter<205:
        
        batch_x = _TRAINING_INPUT[batch_init: limit]
        batch_y = _TRAINING_OUTPUT[batch_init: limit]
        batch_init = batch_init + batch_size
        limit = batch_init + batch_size

        batch_x=batch_x.reshape((batch_size,time_steps,input_size))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            print(outputs[-1])
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
        
    #calculating test accuracy
    test_data = _TESTING_INPUT.reshape((-1, time_steps, input_size))
    test_label = _TESTING_OUTPUT
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))
    #print("Prediction:", sess.run(prediction, feed_dict={x: test_data, y: test_label}))
    #d = prediction.eval(feed_dict={x: test_data}, session=sess)
    #print ("predictions", d)
    #print(type(d[0]))

Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  10
Accuracy  0.01
Loss  3.04814
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  20
Accuracy  0.3
Loss  2.62893
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  30
Accuracy  0.64
Loss  1.84877
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  40
Accuracy  0.86
Loss  1.52903
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  50
Accuracy  0.9
Loss  1.03908
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  60
Accuracy  0.93
Loss  0.775554
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell/mul_11:0", shape=(?, 20), dtype=float32)
For iter  70
Accuracy  0.87
Loss  1.36215
__________________
Tensor("rnn/rnn/rnn/basic_lstm_cell

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
# The code was removed by DSX for sharing.